### Importing all the necessary modules for the application.

__*IMPORTANT:* After installing the packages, please restart the Jupyter kernal for the changes to take place.__

In [ ]:
import pkgutil

# installing Diffusers package for Stable Diffusion Image Generation.
if pkgutil.find_loader('diffusers') is None:
    %pip install diffusers
else:
    print('diffusers already installed')

# installing PyTorch for Transformers, accelerators and SafeTensors.
if pkgutil.find_loader('torch') is None:
    %pip install torch
else:
    print('torch already installed')

# installing the Invisible WaterMarking package.
if pkgutil.find_loader('invisible_watermark') is None:
    %pip install invisible_watermark
else:
    print('invisible_watermark already installed')

# installing Gradio for the GUI.
if pkgutil.find_loader('gradio') is None:
    %pip install gradio
else:
    print('gradio already installed')

# installing transformers
if pkgutil.find_loader('transformers') is None:
    %pip install transformers
else:
    print('transformers already installed')

# installing Accelerate
if pkgutil.find_loader('accelerate') is None:
    %pip install accelerate
else:
    print('accelerate already installed')

# installing SafeTensors
if pkgutil.find_loader('safetensors') is None:
    %pip install safetensors
else:
    print('safetensors already installed')


# importing packages
import torch
from diffusers import DiffusionPipeline
import gradio as gr

# printing versions
print(torch.__version__)
# print(diffusers.__version__)
print(gr.__version__)


### Loading the "Base" and "Refiner" models of Stable Diffusion XL

we use the *Base* model to generate the image, and use *Refiner* model to refine the generated image by applying post-processing effects, upscalling, etc.

In [2]:
# loading the Base model
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype = torch.float16,
    variant = "fp16",
    use_safetensors = True
)
base.to("cuda")

# loading the Refiner model
refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2 = base.text_encoder_2,
    vae = base.vae,
    torch_dtype = torch.float16,
    variant = "fp16",
    use_safetensors = True
)
refiner.to("cuda")

c:\Users\katra\anaconda3\envs\StabilityAi-Env\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\katra\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

AssertionError: Torch not compiled with CUDA enabled

### Loading the "Upscaler" model

As the name suggests, it upscales or improves the resolution of the generated images.
This is needed as these models typically support very low resolutions (in this case, a resolution of upto 1024x1024px) which might not be very sharp for sharing your AI masterpieces online.

In [ ]:
# loading the upscaler
upscaler = DiffusionPipeline.from_pretrained(
    "stabilityai/sd-x2-latent-upscaler",
    torch_dtype=torch.float16,
    use_safetensors=True
)
upscaler.to("cuda")

### Defining a function that handles the image generation process

We pass in the prompt, image heigth, and width that the user gives, which is then sent to the models for generating the image.

In [ ]:
def image_generator(prompt, negative_prompt, img_height, img_width, 
                    guidance, steps, seed, enable_upscaling, high_noise_fraction):

    generator = torch.Generator(device = "cuda").manual_seed(seed)
    init_image = base(
        prompt = prompt,
        negative_prompt = negative_prompt,
        height = img_height,
        width = img_width,
        guidance_scale = guidance,
        num_inference_steps = steps,
        generator = generator,
        denoising_end = high_noise_fraction,
    ).images

    if enable_upscaling:
        final_image = refiner(
            
        )
        
